# pwntools

- 安装指令：`pip install pwntools`
- 使用场景： 与服务器进行交互，且在Crypto中也能用来绕过pow(proof of work)共识机制、向提供的ip端口发送payload以及快速编写exp。

## connect

In [ ]:
from pwn import *

# 连接到远程服务器
## host+ip 连接
host = '127.0.0.1'
port = 8080
r = remote(host, port, level='debug')
## ssh 连接
user = 'root'
password = '123456'
s = ssh(host=host, port=port, user=user, password=password)
## 本地调试
p = process('./to_be_pwned')
# 下面我们统一使用 conn 表示连接对象。
conn.close() # 关闭连接

## package

p32/p64 能够将哈希值变为字节流并填充至32/64位 bits

In [ ]:
from pwn import *
test_hex = 0xdeadbeef
# attack_hex = 0x1234567890abcdef # 待打包内容字节数不能够超过对应位数
p32(test_hex), p64(test_hex) #, p32(attack_hex), p64(attack_hex)

(b'\xef\xbe\xad\xde',
 b'\xef\xbe\xad\xde\x00\x00\x00\x00',
 b'xV4\x12',
 b'xV4\x12\x00\x00\x00\x00')

## send and receive

最重要的当然是数据交互：

In [ ]:
data = b"I wanna Flag"

conn.send(data) # 发送数据
conn.sendline(data) # 发送数据并换行，相当于添加了 `\n`
conn.sendafter('prompt', data) # 发送数据直到遇到 `prompt`

conn.recv() # 接收数据
conn.recv(n) # 接收 n 字节数据
conn.recvline() # 接收一行数据
conn.recvall(timeout=1) # 接收所有数据，直到 1s 未接收到信息
conn.recvuntil('prompt') # 接收数据直到遇到 `prompt`
conn.recvuntil_match(re.compile(b'prompt')) # 接收数据直到遇到正则表达式

conn.interact() # 交互式操作，相当于进入手动操作

下面这段代码主要用于绕过基于工作量证明（Proof of Work, PoW）的验证机制。

> [!CITE]
>
> [Proof_of_work](https://www.wikiwand.com/en/articles/Proof_of_work)

In [ ]:
import binascii
from pwn import remote
import string
import itertools
from hashlib import sha256
from pwn import *
#定义字母表
table = string.ascii_letters+string.digits
#定义PoW函数
def PoW(hash_value, part):
    alphabet = string.ascii_letters + string.digits
    for x in itertools.product(alphabet, repeat=4):
        nonce = ''.join(x)
        if sha256((nonce + part.decode()).encode()).hexdigest() == hash_value.decode():
        	return nonce

host = '127.0.0.1'
port = 8080
conn = remote(host, port)
msg = conn.recvuntil(b'Plz tell me XXXX:')
print(msg)
part = msg[12:28]
hashvalue = msg[33:-18]
print(part, hashvalue)
nonce = PoW(hashvalue, part)
print(nonce)
conn.sendline(nonce.encode()) # 发送我们的“证明”
conn.recvuntil(b'> ')

In [ ]:
import pwn
help(pwn.ssh)

Help on class ssh in module pwnlib.tubes.ssh:

class ssh(pwnlib.timeout.Timeout, pwnlib.log.Logger)
 |  ssh(user=None, host=None, port=22, password=None, key=None, keyfile=None, proxy_command=None, proxy_sock=None, level=None, cache=True, ssh_agent=False, ignore_config=False, raw=False, *a, **kw)
 |  
 |  Method resolution order:
 |      ssh
 |      pwnlib.timeout.Timeout
 |      pwnlib.log.Logger
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, attr)
 |      Permits function-style access to run commands over SSH
 |      
 |      Examples:
 |      
 |          >>> s =  ssh(host='example.pwnme')
 |          >>> print(repr(s('echo hello')))
 |          b'hello'
 |  
 |  __enter__(self, *a)
 |  
 |  __exit__(self, *a, **kw)
 |  
 |  __getattr__(self, attr)
 |      Permits member access to run commands over SSH
 |      
 |      Examples:
 |      
 |          >>> s =  ssh(host='example.pwnme')
 |          >>> s.echo('hello')
 |          b'hello'
 |          >>>